

# 1. *In silico* generation of chemical structures with OMG

Download PMG:
*PMG is the Parallel Molecules Generator, the version of OMG
that allows us to run it on several cores, I use the name OMG later as well but always mean this program.

choose PMG.zip and unpack
https://sourceforge.net/projects/openmg/files/

In the folder specified below as PMGdir, save phosphate.sdf and organophosphate.sdf. These contain the inital fragments.
The initial fragments for OMG can be generated e.g. by drawing the molecules in ChemDraw and saving them as sdf.

The program requires java to be installed, on windows it must be 'Java x86 32bits'


## packages and working directory

In [1]:
import os
import subprocess
import pandas as pd
import numpy
import math
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from itertools import groupby

In [ ]:
#specify here a path to the folderyou want to work in (for input and output data etc)
Projectdir = "C:\\Users\\Hannah\\Documents\\Master UvA\\master project\\OMG"
#specify here the path where PMG is installed 
PMGdir = "C:\\Users\\Hannah\\OMG PMG\\PMG"
os.chdir(Projectdir)
print("Current Working Directory: " , os.getcwd())

### First thing to try: run PMG

In [3]:
#change wd to PMG directory
os.chdir(PMGdir)
print(os.getcwd())

C:\Users\Hannah\OMG PMG\PMG


In [4]:
#get readme of OMG: 
!java -jar PMG_1.0.jar

Usage: java -jar PMG.jar <formula> [options]

Providing a formula for the elemental compositoin is obligatory, e.g., C4H7NO3.
You can further specify the following options.
	-filter 	Filter bad substructures in the molecular structure.
	-fr 	A file containing one substructure to use as initial structure for generation
	-goodlist	A file containing required substructures of the molecule (checked in the end) - only active if cdk is used
	-badlist	A file containing forbidden substructures (checked in the end) - only active if cdk is used
	-hashmap	Enables using a hashmap with semi-canonicity instead of the minimizer
	-nocdk 	Disables using CDK for removing unacceptable molecular structures in the end.
	-p  	The number of parallel threads to use; by default will use all available cores
	-o  	The name of the output file
	-v  	verbose mode
	-m 	The method to use: 
			0=semi-canonicity; 
			1=minimization; 
			2=canonical-augmentation;
Note that if you don't specify a method (-m) then the mix 

In [5]:
#run a first trial of the program, creating molecules for the sumformula C5H13O4P1, 
#save as .txt, with initial fragment organophosphate
!java -jar PMG_1.0.jar C5H13O4P1 -o trial_1.txt -fr organophosphate.sdf

Processing C5H13O4P1 using semi-canonization and minimization test on every generated structure
Selected options: cdk, with output to file, 4 threads.
Using the first molecule in organophosphate.sdf as the starting fragment.

Final molecule count:  3
Duration: 1418 milliseconds


## Functions for Workflow to generate Molecules

In [3]:
#####################     build table from mol -> sumformula -> table    #############################

def formula(mol):
    '''Returns the chemical formula of a molecule'''
    return(Chem.rdMolDescriptors.CalcMolFormula(mol))

def numberofElement(formula, Element) :
    '''returns number of atoms of element in sumformula'''
    #make sum formula list from SMILES, separated into elements and numbers
    Sumformula = [sub for i, j in groupby(list(formula), str.isdigit) 
                  for sub in ([''.join(j)] if i else j)] 
    #this is necessary for two letter elements:
    Sumformula = [sub for i, j in groupby(Sumformula, str.isalpha) 
                  for sub in ([''.join(j)] if i else j)] 
    #find index of Element number
    if Element in Sumformula:
        index = Sumformula.index(Element) +1
        #if element not included return
    else: return(numpy.nan)
    #give number of element
    try: 
        nElement = Sumformula[index]
        return(int(nElement))
    #in case of one element, no number is found, nElement is then 1
    except IndexError:
        #double check Sumformula[index-1]
        nElement = 1
        return(int(nElement))


def makeElementsTable (Sumformula):
    '''creates element table from sumformula'''
    Elementslist = ['C','H','O','N','P','S','Si','Cl','Br','F']
    #first column will be the sumformula, then come then elements as specified here
    data = {
        'C':numberofElement(Sumformula,'C'),
        'H':numberofElement(Sumformula,'H'),
        'O':numberofElement(Sumformula,'O'),
        'N':numberofElement(Sumformula,'N'),
        'P':numberofElement(Sumformula,'P'),
        'S':numberofElement(Sumformula,'S'),
        'Si':numberofElement(Sumformula,'Si'),
        'Cl':numberofElement(Sumformula,'Cl'),
        'Br':numberofElement(Sumformula,'Br'),
        'F':numberofElement(Sumformula,'F')}
    Sumformulas = pd.DataFrame([data], columns = Elementslist)
    #check if there ale elements in the sumformula, for which there are no columns:
    x = 0
    for i in Sumformula:
        #for all elements in sumformula if they are letters
        if i.isalpha():
            if i not in Elementslist :
                #x gets raised to 1 if the warning is printed
                x = 1
                print('Warning: Elements which are not contained in table appeared in input formula!')
    #if the above statement was not executed no extra elements are found and x remained 0
    if x == 0:
        print('Check: All elements in sumformula covered in table :)')
    return(Sumformulas)


#---------------------operators for element table------------------------------------------

def addAtomX(df,X,nx=1, Y='non', ny=1):
    '''Element X, number of Atoms of Element X nx, Element Y, number of Atoms of Element Y yn,
    adds specified number of atoms of one or two specified elements to element table'''
    #make a new row, copy last row from df and increase value in cell of Element X by nx
    newrow = df.iloc[[-1]].reset_index(drop=True)
    newrow.at[0,X]+= nx
    #if second Element specified, increase value in cell of Element Y by ny
    if Y != 'non':
        newrow.at[0,Y]+= ny
    #append newrow to dataframe
    newdf = df.append(newrow,ignore_index=True)
    return(newdf)

def addnAtoms(df,times,X, nx=1, Y='non', ny=1):
    '''add Element/combination of Elements times times, loop to add atoms, creates times new rows'''
    for i in range(times):
        df = addAtomX(df,X, nx, Y, ny)
    return(df)

def addCH2(df,times):
    '''adds times rows increased by one CH2 unit'''
    df=addnAtoms(df,times,'C',1, 'H',2)
    return(df)

#test
ElementTable1 = makeElementsTable('C3H9O4P')

#print(ElementTable1,'\n')
ElementTable2 = addCH2(ElementTable1,3*9)
print(ElementTable2,'\n')
#ElementTable3 = addnAtoms(ElementTable1,3,'C',1,'H',2)
#print(ElementTable3)

#-------------------------------------------------------------------------

def makeformulalist(df):
    '''turn table rows in list of strings of sumformulas'''
    #empty list to store formulas in:
    allformulas = []
    #iterate through rows:
    for row in range(len(df)):
        #create empty string to store sumformula in:
        formula = ''
        #iterate through Elements, which are the column names:
        for Element in df.columns:     
            #check if cell is not empty:
            if math.isnan(df[Element][row])!=True:
                #if there is a number in the cell of row and Element, concetenate Element with string of cell number 
                #append element and number to formula string
                #print(Element + str(ElementTable3[Element][row]))
                formula += Element + str(df[Element][row])
        #print(formula)
        #append formula to list of allformulas
        allformulas.append(formula)
    return(allformulas)


###################    OMG     ############################################################


def runOMG(formulalist, outname, fragment, good = "", bad = "", additionalcmd =""):
    '''runs OMG for all formulas in formulalist and saves output from every fomula in numbered outname files, 
    returns dictionary of sumformula keys and corresponding filenames as values'''
    #set directory to PMG
    os.chdir(PMGdir)
    #number for files, create empty dictionary to store files in
    nr = 0
    filenames = {}
    for formula in formulalist:
        #make java command
        command = "java -jar PMG_1.0.jar " + formula + " -o " + str(outname) + str(nr) + ".txt -fr " + fragment +".sdf"+ good +bad + additionalcmd
        print(command)
        #run OMG
        p1 = subprocess.run(str(command),shell=True, capture_output=True, text=True)
        #print OMG communication
        print(p1.stdout)
        #in case of error print OMG error and assign error value to corresponding formula in dictionary
        if p1.returncode != 0:
            filenames[formula] = 'Error'
            print('-> OMG Error:',p1.stderr)
        #if execution properly, enter formula and filename in dictionary and increase counter nr by 1
        else:
            filenames[formula] = (str(outname) + str(nr)+'.txt')
            nr += 1
    print('OMG finished, returned this dictionary: \n',filenames)
    return(filenames)

#try out error statement with this list: formulalist4= ['C3H9O4P', 'C4H11O4P1X', 'C5H13O4P1', 'C6H15O4P1R']
#test: omgout1 = runOMG(formulalist3,'newww', 'phosphate')


###########################################    sdf to SMILES.txt   ###########################


def SDftoSMILESlist(inSDF, infolder = PMGdir):
    '''coverts SDF to list of kekulized smiles'''
    #import molecules from SD files
    mols = Chem.SDMolSupplier((str(infolder)+"\\"+str(inSDF)))
    #print(Chem.MolToSmiles(mols[1]))
    #create empty list to store smiles in
    SMILESlist = []
    #iterate trough molecules, convert to smiles and append to list of smiles
    i = 0
    for mol in mols:
        try: 
            Chem.Kekulize(mol)
            out = SMILESlist.append(Chem.MolToSmiles(mol,kekuleSmiles=True))
        except:
            i += 1
    print(i, 'mols from OMGoutput could not be read')
    return(SMILESlist)

def writetxt(alist, name, outfolder = os.getcwd()):
    '''writes a list to a txt with every element in list in next line, input name without ending'''
    with open(os.path.join(outfolder,(str(name)+'.txt')), 'w') as f:
        for item in alist:
            f.write("%s\n" % item)

def makeSmilestxtsfromOMGout(OMGoutdict):
    '''creates smiles lists from .sdf output of OMG, writes one file per formula with filename formula.txt 
    into SMILESOMGout projectfolder'''
    #set directory to PMG
    os.chdir(PMGdir)
    #iterate through dictionary
    i= 1
    for formula, file in OMGoutdict.items():
        #print i. Error for every error found and thus skip this key pair
        if file == 'Error':
            print(i,'. Error')
            i += 1
        #for all others:
        else:
            #convert .sdfs in files to smiles list, function defined above
            smiles = SDftoSMILESlist(file)
            #writes txt from smileslist with formula as filename into SMILESOMGout projectfolder, 
            #function defined above
            try:
                writetxt(smiles,formula, Projectdir + '\\SMILESOMGout')
            #if error occurs, try again after creating the directory
            except:
                os.mkdir(Projectdir + '\\SMILESOMGout')
                writetxt(smiles,formula, Projectdir + '\\SMILESOMGout')
    print('All SMILES.txt\'s in', Projectdir + '\\SMILESOMGout', '\n', OMGoutdict)
            

def jointxts(filenameslist, outname, outfolder=(Projectdir + '\\SMILESOMGout'),
             infolder=(Projectdir + '\\SMILESOMGout')):
    '''joins txts whose names are specified in a list into on txt with specified name. Folder where to find txts 
    and folder where to store joined txt can be specified '''
    #filenameslist is list of files in infolder, outname is name of outputfile in outfolder
    with open(os.path.join(outfolder,(str(outname)+'.txt')), 'w') as outfile:
        for fname in filenameslist:
            with open(os.path.join(infolder, (str(fname)+'.txt'))) as infile:
                for line in infile:
                    outfile.write(line)

def delErrordict(dict):
    '''Returns dictionary without pairs that contain Error'''
    newdict = {k: v for k, v in dict.items() if v != 'Error'}
    return(newdict)

def OMGoutdicttojoinedtxt(OMGoutdict, allname, outfolder=(Projectdir + '\\SMILESOMGout'),
             infolder=(Projectdir + '\\SMILESOMGout')):
    '''Joins output form OMG as specified in output dictionary and stores that with specified name.
    Folder where to find txts and folder where to store joined txt can be specified '''
    #OMGoutdict = delErrordict(OMGoutdict)
    jointxts(list(OMGoutdict.keys()),allname,outfolder,infolder)
    print('Joined SMILES.txt in',outfolder,' as ',allname)


Check: All elements in sumformula covered in table :)
     C   H  O   N  P   S  Si  Cl  Br   F
0    3   9  4 NaN  1 NaN NaN NaN NaN NaN
1    4  11  4 NaN  1 NaN NaN NaN NaN NaN
2    5  13  4 NaN  1 NaN NaN NaN NaN NaN
3    6  15  4 NaN  1 NaN NaN NaN NaN NaN
4    7  17  4 NaN  1 NaN NaN NaN NaN NaN
5    8  19  4 NaN  1 NaN NaN NaN NaN NaN
6    9  21  4 NaN  1 NaN NaN NaN NaN NaN
7   10  23  4 NaN  1 NaN NaN NaN NaN NaN
8   11  25  4 NaN  1 NaN NaN NaN NaN NaN
9   12  27  4 NaN  1 NaN NaN NaN NaN NaN
10  13  29  4 NaN  1 NaN NaN NaN NaN NaN
11  14  31  4 NaN  1 NaN NaN NaN NaN NaN
12  15  33  4 NaN  1 NaN NaN NaN NaN NaN
13  16  35  4 NaN  1 NaN NaN NaN NaN NaN
14  17  37  4 NaN  1 NaN NaN NaN NaN NaN
15  18  39  4 NaN  1 NaN NaN NaN NaN NaN
16  19  41  4 NaN  1 NaN NaN NaN NaN NaN
17  20  43  4 NaN  1 NaN NaN NaN NaN NaN
18  21  45  4 NaN  1 NaN NaN NaN NaN NaN
19  22  47  4 NaN  1 NaN NaN NaN NaN NaN
20  23  49  4 NaN  1 NaN NaN NaN NaN NaN
21  24  51  4 NaN  1 NaN NaN NaN NaN NaN
22 

## Workflow

In [ ]:
####################template##########################
Sumformula = #specify starting formula

#Sumformula -> Elements df
makeElementsTable(Sumformula)

#Elements df <- new rows
addAtomX(df,X,nx=1, Y='non', ny=1)
addnAtoms(df,times,X, nx=1, Y='non', ny=1)
addCH2(df,times)

#Elements df -> list of formulas
makeformulalist(df)

#List of formulas -> file per formula with structures(sdf)
#-> dictionary of formulas:filename
runOMG(formulalist, outname, fragment)

#if dictionary of formulas:filename contains errors, delete them
#-> dictionary of formulas:filename without errors
delErrordict(dict)

#dictionary of formulas:filename -> smiles.txt per formula
makeSmilestxtsfromOMGout(OMGoutdict)

#dictionary of formulas:filename, [smiles.txt per formula] -> one .txt of smiles
OMGoutdicttojoinedtxt(OMGoutdict, allname, outfolder=(Projectdir + '\\SMILESOMGout'),
             infolder=(Projectdir + '\\SMILESOMGout'))

try:
    os.mkdir(Projectdir + '\\SMILESmodelin')
except(FileExistsError):
    print('SMILESmodelin folder exists already')
OMGoutdicttojoinedtxt(omgout1,'1',(Projectdir + '\\SMILESmodelin'))

## Next thing: Creating series by adding CH2 units
This gives us a list of SMILES of structures with extended alkyl chains.

In [13]:
#series 1, alkylextension
#Workflow for alkyl variation
#set parameters:
sumS = 'C4H11O4P' #starting formula -> #startformula() to promt input
fragment = 'organophosphate' #initial fragment input to OMG
OMGgoodcmd = "" #good list argument to OMG, not used here, as these are applied only as post-calculation filters
OMGbadcmd = ""
OMGadditional = ""
outnameOMG = 'S2OMG' #name to store OMG output
allsmilesname = 'allsmilesS2' #name of file in which all SMILES together are stored
ToaddCH2 = True #Boolean to switch CH2 unit addition on (True) or off (False)
timesCH2 = 3 #number of CH2 units to add

#Sumformula -> Elements df
ElementstableS = makeElementsTable(sumS)

#Elements df <- new rows
#addAtomX(df,X,nx=1, Y='non', ny=1)
#addnAtoms(df,times,X, nx=1, Y='non', ny=1)
if ToaddCH2:
    ElementstableSadded = addCH2(ElementstableS,timesCH2)
    print(ElementstableSadded)
else: ElementstableSadded = ElementstableS

#Elements df -> list of formulas
formulalistS = makeformulalist(ElementstableSadded)

#List of formulas -> file per formula with structures(sdf)
#-> dictionary of formulas:filename
OMGoutdictS = runOMG(formulalistS, outnameOMG, fragment, OMGgoodcmd, OMGbadcmd, OMGadditional)

#if dictionary of formulas:filename contains errors, delete them
#-> dictionary of formulas:filename without errors
OMGoutdictS = delErrordict(OMGoutdictS)

#dictionary of formulas:filename -> smiles.txt per formula
makeSmilestxtsfromOMGout(OMGoutdictS)

#dictionary of formulas:filename, [smiles.txt per formula] -> one .txt of smiles
OMGoutdicttojoinedtxt(OMGoutdictS, allsmilesname,(Projectdir + '\\SMILESmodelin'))
                      
                      
                      
                      

Check: All elements in sumformula covered in table :)
   C   H  O   N  P   S  Si  Cl  Br   F
0  4  11  4 NaN  1 NaN NaN NaN NaN NaN
1  5  13  4 NaN  1 NaN NaN NaN NaN NaN
2  6  15  4 NaN  1 NaN NaN NaN NaN NaN
3  7  17  4 NaN  1 NaN NaN NaN NaN NaN
java -jar PMG_1.0.jar C4H11O4P1 -o S2OMG0.txt -fr organophosphate.sdf
Processing C4H11O4P1 using semi-canonization and minimization test on every generated structure
Selected options: cdk, with output to file, 32 threads.
Using the first molecule in organophosphate.sdf as the starting fragment.

Final molecule count:  1
Duration: 867 milliseconds

java -jar PMG_1.0.jar C5H13O4P1 -o S2OMG1.txt -fr organophosphate.sdf
Processing C5H13O4P1 using semi-canonization and minimization test on every generated structure
Selected options: cdk, with output to file, 32 threads.
Using the first molecule in organophosphate.sdf as the starting fragment.

Final molecule count:  3
Duration: 867 milliseconds

java -jar PMG_1.0.jar C6H15O4P1 -o S2OMG2.txt -fr o

## Example run: Add one oxygen, run alkyl extension

In [14]:
#series 1, alkylextension
#Workflow for alkyl variation
sumS = 'C4H11O5P' #starting sumformula -> #startformula() to promt input
fragment = 'organophosphate' #initial fragment input to OMG
OMGgoodcmd = ""
OMGbadcmd = ""
OMGadditional = ""
outnameOMG = 'OMG0616_SO_1' #name to store OMG output
allsmilesname = 'allsmiles0616_SO_1' #name of file in which all SMILES together are stored
ToaddCH2 = True #Boolean to switch on or off
timesCH2 = 15 #number of CH2 units to add

#Sumformula -> Elements df
ElementstableS = makeElementsTable(sumS)

#Elements df <- new rows
#addAtomX(df,X,nx=1, Y='non', ny=1)
#addnAtoms(df,times,X, nx=1, Y='non', ny=1)
if ToaddCH2:
    ElementstableSadded = addCH2(ElementstableS,timesCH2)
    print(ElementstableSadded)
else: ElementstableSadded = ElementstableS

#Elements df -> list of formulas
formulalistS = makeformulalist(ElementstableSadded)

#List of formulas -> file per formula with structures(sdf)
#-> dictionary of formulas:filename
OMGoutdictS = runOMG(formulalistS, outnameOMG, fragment, OMGgoodcmd, OMGbadcmd, OMGadditional)

#if dictionary of formulas:filename contains errors, delete them
#-> dictionary of formulas:filename without errors
OMGoutdictS = delErrordict(OMGoutdictS)

#dictionary of formulas:filename -> smiles.txt per formula
makeSmilestxtsfromOMGout(OMGoutdictS)

#dictionary of formulas:filename, [smiles.txt per formula] -> one .txt of smiles
OMGoutdicttojoinedtxt(OMGoutdictS, allsmilesname,(Projectdir + '\\SMILESmodelin'))

Check: All elements in sumformula covered in table :)
     C   H  O   N  P   S  Si  Cl  Br   F
0    4  11  5 NaN  1 NaN NaN NaN NaN NaN
1    5  13  5 NaN  1 NaN NaN NaN NaN NaN
2    6  15  5 NaN  1 NaN NaN NaN NaN NaN
3    7  17  5 NaN  1 NaN NaN NaN NaN NaN
4    8  19  5 NaN  1 NaN NaN NaN NaN NaN
5    9  21  5 NaN  1 NaN NaN NaN NaN NaN
6   10  23  5 NaN  1 NaN NaN NaN NaN NaN
7   11  25  5 NaN  1 NaN NaN NaN NaN NaN
8   12  27  5 NaN  1 NaN NaN NaN NaN NaN
9   13  29  5 NaN  1 NaN NaN NaN NaN NaN
10  14  31  5 NaN  1 NaN NaN NaN NaN NaN
11  15  33  5 NaN  1 NaN NaN NaN NaN NaN
12  16  35  5 NaN  1 NaN NaN NaN NaN NaN
13  17  37  5 NaN  1 NaN NaN NaN NaN NaN
14  18  39  5 NaN  1 NaN NaN NaN NaN NaN
15  19  41  5 NaN  1 NaN NaN NaN NaN NaN
java -jar PMG_1.0.jar C4H11O5P1 -o OMG0616_SO_10.txt -fr organophosphate.sdf
Processing C4H11O5P1 using semi-canonization and minimization test on every generated structure
Selected options: cdk, with output to file, 32 threads.
Using the first mole

KeyboardInterrupt: 

In [15]:
#Sometimes, fixing was needed, mostly due to 'translation errors' of chemical file formats in the workflow
#if dictionary of formulas:filename contains errors, delete them
#-> dictionary of formulas:filename without errors
OMGoutdictS = delErrordict(OMGoutdictS)

#dictionary of formulas:filename -> smiles.txt per formula
makeSmilestxtsfromOMGout(OMGoutdictS)

#dictionary of formulas:filename, [smiles.txt per formula] -> one .txt of smiles
OMGoutdicttojoinedtxt(OMGoutdictS, allsmilesname,(Projectdir + '\\SMILESmodelin'))

All SMILES.txt's in C:\Users\Hannah\Desktop\OMGfiles\SMILESOMGout 
 {'C4H11O4P1': 'S2OMG0.txt', 'C5H13O4P1': 'S2OMG1.txt', 'C6H15O4P1': 'S2OMG2.txt', 'C7H17O4P1': 'S2OMG3.txt'}
Joined SMILES.txt in C:\Users\Hannah\Desktop\OMGfiles\SMILESmodelin  as  allsmiles0616_SO_1
